In [1]:
import numpy as np
import numpy.testing as npt
from scipy.linalg import expm
import matplotlib.pyplot as plt
from matplotlib.transforms import Affine2D
from matplotlib.collections import PatchCollection
import scipy.spatial, scipy.linalg
import scipy.sparse.linalg
from scipy.misc import logsumexp
from scipy.cluster.hierarchy import linkage, dendrogram
import itertools as it
import time
%matplotlib inline


In [2]:
np.sign(np.random.randn(10))

array([-1.,  1., -1.,  1., -1.,  1.,  1.,  1.,  1., -1.])

In [ ]:
class svm_smo(): 
    
    def __init__(self, kernel, kernelparameter, regularization):
        self.kernel = kernel
        self.kernelparameter = kernelparameter
        self.regularization = regularization
        self.kp = kernelparameter
        self.c = regularization

    def getkernel(self, X, Y=None):
        n= len(X)
        n2 = n
        if(Y==None):
            X2=np.array(X)
            x2=len(X2)
            n2 = n
        else: 
            X2 = Y
            n2 = len(X2)
 
        if((self.kernel =='gaussian') or (self.kernel ==['gaussian'])):
            w = self.kp
            X1 = (X**2).sum(1).reshape(n,1)*np.ones((n,n2))
            U1 = (X2**2).sum(1).reshape(1,n2) * np.ones([n,n2])
            D = X1 - 2*(X.dot(X2.T)) + U1
            K = np.exp(-D/(2*w**2))
        elif((self.kernel =='polynomial') or (self.kernel ==['polynomial'])):
            p= self.kp
            K = (np.dot(X,X2.T)+1)**p
        elif((self.kernel =='linear') or (self.kernel ==['linear'])):
            K = np.dot(X,X2.T)
        else:
            raise AssertionError("Choose from ['gaussian','polynomial','linear']")
        return K
    
    
    def fx(self,X1,X2,Y):
        K=getkernel(X1,X2)
        alpY = (Y*self.alpha).reshape(len(self.alpha),1)
        return np.sign(np.dot(K,alpY)+self.b)
    
    def _compute_box_constraints(self, i, j, Y, alpha, C):
        blidx=(Y[i]==Y[j])
        L=np.zeros(len(i))
        H=np.zeros(len(i))
        L[np.where(blidx==1)] = (alpha[i]+alpha[j]-C)[np.where(blidx==1)]
        L[np.where(blidx==0)] =(alpha[j]-alpha[i])[np.where(blidx==0)]
        L[np.where(L<0)] = 0
        H[np.where(blidx==1)] = (alpha[i]+alpha[j])[np.where(blidx==1)]
        H[np.where(blidx==0)] =(C+alpha[j]-alpha[i])[np.where(blidx==0)]
        H[np.where(H>C)] = C
        return L, H 
    
    
    def _update_parameters(self, E_i, E_j, i, j, K, Y, alpha, b, C):
        L, H = self._compute_box_constraints(self, i, j, Y, alpha, C)
        alpha_old =  np.array(alpha)
        alpha_new =  np.array(alpha)
        n= np.arange(len(i))
        nochangeidx=np.where((L==H)==1)
        kappa = 2*K[i,j] -K[i,i]-K[j,j]
        nochangeidx2 = np.where(kappa>=0)
        n=np.setdiff1d(n,nochangeidx)
        
        if(len(n)==0):
            return alpha_old, b, False
        changeidx=np.setdiff1d(n,nochangeidx2)
        if(len(changeidx)==0):
            return alpha_old, b, False
        
        alphaj_old=np.array(alpha[j[changeidx]])
        alpha[j[changeidx]]= alpha[j[changeidx]]-Y[j[changeidx]]*(E_i[changeidx]-E_j[changeidx])/kappa[i[changeidx]]
        pointa= (alpha[j])[changeidx]
        pointa[np.where(poita>H[changeidx])]=(H[changeidx])[np.where(poita>H[changeidx])]
        pointa[np.where(poita<L[changeidx])]=(H[changeidx])[np.where(poita<L[changeidx])]
        alpha[i[changeidx]]= alpha[i[changeidx]]-Y[i[changeidx]]*Y[j[changeidx]]*(alphaj_old-alpha[j[changeidx]])
        chg=np.where(np.abs(alphaj_old-alpha[j[changeidx]])>0.0005)
        if(len(chg)==0):
            return alpha_old, b, False
        newchgidx=changeidx[chg]
        alpha_new[i[newchgidx]]=alpha[i[newchgidx]]
        return alpha, new_b, updated
    
    
    def _compute_updated_b(self, E_i, E_j, i, j, K, Y, alpha_old, alpha_new, b_old, C):
        
        return new_b
     
        
    def fit(self, X, y):
        #self.X_fit = X
        n= len(X)
        K = self.getkernel(X)
        self.K = K
        self.alpha = np.zeros(n)
        self.b = 1
        P=100
        tol = 3
        b1 = np.zeros((len(y),1))
        b2 = np.zeros((len(y),1))
        b3 = np.zeros((len(y),1))
        b4 = np.zeros((len(y),1))
        p=0
        
        while(p<P):
            a=0
            E = fx(X,X,y)-y.reshape(len(y),1)
            b1[np.where((E*y.reshape(len(y),1))<(-tol))] = 1
            b2[np.where(self.alpha<self.c)] = 1   #.astype(np.int64)
            b12 = b1*b2
            b3[np.where((E*y.reshape(len(y),1))>tol)] = 1
            b4[np.where(self.alpha>0)] = 1   #.astype(np.int64)
            b34 = b3*b4
            blidx = (b12+b34)
            i = np.where(blidx>0)
            j = np.random.choice(len(self.alpha),len(i),False)
            samebl = (i==j).astype(np.int64)
            check=sum(samebl)
            while(check>0):
                bb=np.random.choice(np.setdiff1d(len(self.alpha), j[np.where(samebl==0)]),check,False)
                j[np.where(samebl==1)]=bb
                samebl = (i==j).astype(np.int64)
                check=sum(samebl)
                
            new_alpha, new_b, updated=self._update_parameters(E[i], E[j], i, j, K, y, self.alpha, b, self.c)
            
            p=p+1

        KK = self.K + self.c *np.eye(n)+np.eye(n)*0.000000001
        inv = np.linalg.solve(KK,np.eye(n))
        self.alpha= np.dot(inv,y.reshape(n,1))
        
    def predict(self, X):
        K = self.getkernel(X, self.X_fit)
        return np.dot(K, self.alpha)

In [146]:
a=np.array([1,3,4,5,6,7,9,10,12,13,14,19,20])
rang =np.arange(20)
b=np.random.choice(rang ,len(a),False)
samebl = (a==b).astype(np.int64)
check=sum(samebl)
print(a)
print(b)
print(check)
while(check>0):
    bb=np.random.choice( np.setdiff1d(rang, b[np.where(samebl==0)]),check,False)
    b[np.where(samebl==1)]=bb
    samebl = (a==b).astype(np.int64)
    check=sum(samebl)
    print(a)
    print(b)
    print(check)

[ 1  3  4  5  6  7  9 10 12 13 14 19 20]
[11 18 15  2  4 17  9 14 13  8  6 10 12]
1
[ 1  3  4  5  6  7  9 10 12 13 14 19 20]
[11 18 15  2  4 17  1 14 13  8  6 10 12]
0


In [170]:
tes = np.arange(0,9)
tes

array([0, 1, 2, 3, 4, 5, 6, 7, 8])

In [171]:
arr = tes[0:4]

In [177]:
arr[2:4]=np.array([9,8])

In [181]:
dif=np.setdiff1d(tes,np.array([1,2,3]))
dif

array([0, 4, 5, 6, 7, 8, 9])

In [183]:
np.setdiff1d(dif,np.array([1,2,3,4]))

array([0, 5, 6, 7, 8, 9])

In [196]:
(tes[0:4])[np.array([1,3])]

array([1, 8])

In [205]:
len(np.setdiff1d(tes,np.arange(10)))

0